In [43]:
import pandas as pd
import time
from geopy.geocoders import Nominatim 

def geocode_country(country_name):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(country_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None


def data_format(df):
    df['Connections'] = df['Connections'].astype(int)
    df = df.drop(['Population'], axis=1)
    for index, row in df.iterrows():
        row['Latitude'], row['Longitude'] = geocode_country(row['Country Name'])
    index_names = df[ (df['Connections'] < 1000)].index  
    df.drop(index_names, inplace = True)
    df.reset_index(inplace = True) 
    return df


def __main__():
    csv_file = '/home/dipp/Github/Master-Thesis-dipp/Geo_Map_Final/Data/Country_Data_V2_Latest.csv'
    data = pd.read_csv(csv_file)
    data = data_format(data)
    return data


if __name__=="__main__":
    start_time = time.time()
    __main__()
    print("--- %s seconds ---" % (time.time() - start_time)) 

--- 75.47163391113281 seconds ---


In [44]:
data = __main__()

In [45]:
data.at[56, 'Latitude'] = 31.50000000
data.at[56, 'Longitude'] = 34.75000000

In [46]:
print(data.loc[data['Country Code'] == 'IL'])

    index Country Code Country Name  Connections  Latitude  Longitude
56     58           IL       Israel        29758      31.5      34.75


In [47]:
data = data.drop(['index'], axis=1)
data

,Country Code,Country Name,Connections,Latitude,Longitude
0,CA,Canada,1541103,61.066692,-107.991707
1,US,United States,35310671,39.783730,-100.445882
2,IN,India,5133941,22.351115,78.667743
3,ES,Spain,203364,39.326068,-4.837979
4,AU,Australia,830736,-24.776109,134.755000
...,...,...,...,...,...
92,DJ,Djibouti,3584,11.814597,42.845306
93,BJ,Benin,6022,9.529347,2.258441
94,TN,Tunisia,1310,33.843941,9.400138
95,HU,Hungary,1289,47.181759,19.506094


In [48]:
data

,Country Code,Country Name,Connections,Latitude,Longitude
0,CA,Canada,1541103,61.066692,-107.991707
1,US,United States,35310671,39.783730,-100.445882
2,IN,India,5133941,22.351115,78.667743
3,ES,Spain,203364,39.326068,-4.837979
4,AU,Australia,830736,-24.776109,134.755000
...,...,...,...,...,...
92,DJ,Djibouti,3584,11.814597,42.845306
93,BJ,Benin,6022,9.529347,2.258441
94,TN,Tunisia,1310,33.843941,9.400138
95,HU,Hungary,1289,47.181759,19.506094


In [18]:
import folium
import pandas as pd


# Create a map centered on a specific location (you can adjust the coordinates and zoom level)
m = folium.Map(location=[20, 0], zoom_start=2)

# Add markers for each country with bubble labels for country name and connections
for index, row in data.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=10,  # Adjust the size of the bubble
        color='blue',  # Adjust the color of the bubble
        fill=True,
        fill_color='blue',  # Adjust the color of the inside of the bubble
    ).add_to(m)
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=folium.DivIcon(html=f"<div>{row['Country Name']}<br>{row['Connections']} Connections</div>")
    ).add_to(m)

# Save the map to an HTML file
m.save('world_map_with_bubbles.html')


In [19]:
import folium
import pandas as pd



# Create a folium map centered around the world
world_map = folium.Map(location=[0, 0], zoom_start=2)

# Add markers for each country with just the Country Name and Connections displayed
for index, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=folium.DivIcon(
            icon_size=(150,36),
            html=f'<div><b>{row["Country Name"]}</b><br>Connections: {row["Connections"]}</div>'
        )
    ).add_to(world_map)

# Save the map to an HTML file
world_map.save('world_map_01.html')


In [10]:
import geopandas as gpd

gdf = gpd.read_file('/home/dipp/Github/Master-Thesis-dipp/GEO_map_V2/Tutorial/World_Countries_(Generalized)/World_Countries__Generalized_.shp')

gdf.head()

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Leng,SHAPE_Area,geometry
0,1,Afghanistan,AF,Afghanistan,AF,50.803210,62.566714,"POLYGON ((61.27655 35.60725, 61.29638 35.62854..."
1,2,Albania,AL,Albania,AL,9.625038,3.073540,"POLYGON ((19.57083 41.68527, 19.58195 41.69569..."
2,3,Algeria,DZ,Algeria,DZ,70.540082,213.513461,"POLYGON ((4.60335 36.88791, 4.63555 36.88638, ..."
3,4,American Samoa,AS,United States,US,0.600124,0.013720,"POLYGON ((-170.74390 -14.37555, -170.74942 -14..."
4,5,Andorra,AD,Andorra,AD,0.937352,0.055585,"POLYGON ((1.44584 42.60194, 1.48653 42.65042, ..."


In [13]:
world_gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude), crs="EPSG:4326"
)
world_gdf

,Country Code,Country Name,Connections,Latitude,Longitude,geometry
0,CA,Canada,1541103,61.066692,-107.991707,POINT (-107.99171 61.06669)
1,US,United States,35310671,39.783730,-100.445882,POINT (-100.44588 39.78373)
2,IN,India,5133941,22.351115,78.667743,POINT (78.66774 22.35111)
3,ES,Spain,203364,39.326068,-4.837979,POINT (-4.83798 39.32607)
4,AU,Australia,830736,-24.776109,134.755000,POINT (134.75500 -24.77611)
...,...,...,...,...,...,...
92,DJ,Djibouti,3584,11.814597,42.845306,POINT (42.84531 11.81460)
93,BJ,Benin,6022,9.529347,2.258441,POINT (2.25844 9.52935)
94,TN,Tunisia,1310,33.843941,9.400138,POINT (9.40014 33.84394)
95,HU,Hungary,1289,47.181759,19.506094,POINT (19.50609 47.18176)


In [14]:
world_gdf.sort_values(by='Connections', ascending=False)

,Country Code,Country Name,Connections,Latitude,Longitude,geometry
1,US,United States,35310671,39.783730,-100.445882,POINT (-100.44588 39.78373)
2,IN,India,5133941,22.351115,78.667743,POINT (78.66774 22.35111)
10,GB,United Kingdom,3785031,54.702354,-3.276575,POINT (-3.27658 54.70235)
0,CA,Canada,1541103,61.066692,-107.991707,POINT (-107.99171 61.06669)
22,CO,Colombia,1431878,4.099917,-72.908813,POINT (-72.90881 4.09992)
...,...,...,...,...,...,...
94,TN,Tunisia,1310,33.843941,9.400138,POINT (9.40014 33.84394)
95,HU,Hungary,1289,47.181759,19.506094,POINT (19.50609 47.18176)
64,CF,Central African Republic,1281,7.032360,19.998123,POINT (19.99812 7.03236)
96,QA,Qatar,1276,25.333698,51.229529,POINT (51.22953 25.33370)


In [49]:
world_map= folium.Map(tiles="cartodbpositron", zoom_start=10)

In [50]:
%%time

m = folium.Map(tiles='cartodbpositron', zoom_start=20)

#marker_cluster = MarkerCluster().add_to(m)

for index,row in data.iterrows():
    count = row['Connections']
    lat = row['Latitude']
    lon = row['Longitude']
    radius = 30
    popup_text = count



    folium.CircleMarker(location = [lat, lon], radius=radius, tooltip=count, color="cornflowerblue", stroke=False, fill=True, fill_opacity=0.6, opacity=1).add_to(m)

    folium.map.Marker( location = [lat, lon], icon=folium.DivIcon(icon_size=(150,36),icon_anchor=(0,0),html='<div style="font-size: 7pt", font-weight: 900>%s</div>' %count), popup= popup_text).add_to(m)

    #MarkerCluster(locations=[lat, lon], popups=count).add_to(m)

    #folium.Marker(location = [lat, lon]).add_to(m)
    #folium.Marker(location = [lat, lon],icon=folium.DivIcon(html=f"""<div font-weight: bold, style="color: {'green'};">{count}</div>"""),popup= popup_text).add_to(m)

m

CPU times: user 19.7 ms, sys: 0 ns, total: 19.7 ms
Wall time: 19.8 ms


In [28]:
m.save('test.html')